C4 MNIST HW Part 2: PCA images data set, save pca model for further usage

Data Source: https://www.kaggle.com/c/state-farm-distracted-driver-detection

State Farm Distracted Driver Images

10 different labels/categories for each images

Each color image is 640*480 22424 images in training dataset


In [1]:
import pandas as pd
import numpy as np
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
%matplotlib inline

In [2]:
# PIL => Python Imaging Library
from PIL import Image

In [3]:
import os

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
def readcsvfile(filename):
    file_open=open(filename,'r')
    lines_data=[]
    for line in iter(file_open):
        if 'subject' in line:
            continue
        line=line.rstrip()
        lines_data.append(line.split(','))
    return lines_data

In [6]:
#'../dataSetImage/stateFarm_distractedDriver_all/imgs/train/c0/img_44733.jpg'
def hasThisFile(filename):
    return os.path.isfile(filename)

In [7]:
def getJPGfileData(filename):
    #print(filename)
    if hasThisFile(filename):
        #print('hasfile')
        img=Image.open(filename)
        #print(img.size)
        img=img.convert('L')
        return list(img.getdata())
        #print(len(img))
        #print(img.size)

In [8]:
def getJPGfileDataStr(filename):
    if hasThisFile(filename):
        img=Image.open(filename)
        img=img.convert('L')
        tplst=list(img.getdata())
        return ','.join(map(str,tplst))

In [9]:
# under test imges folders, 79,726 images
# under train images folders, 22,424 images
#C:\Users\Chiachi\My Class\Python4ML\HWnProject\dataSetImage\stateFarm_distractedDriver_all
dtsinfo=readcsvfile('../dataSetImage/stateFarm_distractedDriver_all/driver_imgs_list.csv')

In [10]:
# had tried to 5606 size, still not working on np.array(...)
train1, train2 = train_test_split(dtsinfo, test_size=0.95)

In [12]:
print("Original Data Set sample size vs train 1 vs train 2:\n------------------:")
print(len(dtsinfo),len(train1),len(train2))
print(dtsinfo[:3])

Original Data Set sample size vs train 1 vs train 2:
------------------:
22424 1121 21303
[['p002', 'c0', 'img_44733.jpg'], ['p002', 'c0', 'img_72999.jpg'], ['p002', 'c0', 'img_25094.jpg']]


In [14]:
# list of each labels sample size in train1 data set
tpc=[0,0,0,0,0,0,0,0,0,0,0]
for x in train1:
    i=int(x[1][1])
    tpc[i]+=1
print(tpc)

[142, 122, 110, 110, 129, 104, 134, 93, 73, 104, 0]


In [15]:
trainImgFileDir='../dataSetImage/stateFarm_distractedDriver_all/imgs/train/'

In [16]:
testImgFileDir='../dataSetImage/stateFarm_distractedDriver_all/imgs/test/'

In [17]:
XRawTrain1=[]

In [18]:
Ytrain1=[]

In [19]:
SubjectTrain1=[]

In [20]:
for itm in train1:  
    XRawTrain1.append(getJPGfileData(trainImgFileDir+itm[1]+'/'+itm[2]))

In [21]:
Xtrain1NP=np.array(XRawTrain1)

In [22]:
print(Xtrain1NP.shape)

(1121, 307200)


In [23]:
del [[XRawTrain1]]
gc.collect()

0

In [24]:
from sklearn.decomposition import PCA

In [25]:
#640*480 => 16*12
# freeze: 16*12
# 4*3, only cover 0.59
# 8*6 seems better, 0.775127...
# 10*10?, 0.842369
# 12*12, 0.8714058080457765
testpcsz=12*12
colnms=[]
for i in range(1,testpcsz+1):
    colnms.append('pc'+str(i))
print(colnms[:3])

['pc1', 'pc2', 'pc3']


In [26]:
pca = PCA(n_components=testpcsz)

In [27]:
principalComponents = pca.fit_transform(Xtrain1NP)

In [28]:
print(pca.explained_variance_ratio_)

[0.18386806 0.1109155  0.09248646 0.06617043 0.0286103  0.02353063
 0.02130963 0.02032362 0.01733533 0.01526659 0.01305765 0.01205586
 0.01157718 0.01106823 0.00976837 0.00928621 0.00909898 0.00794821
 0.00761247 0.00731967 0.00687491 0.00633913 0.00601527 0.00555999
 0.00500538 0.00486402 0.00452077 0.00445889 0.00435075 0.00402212
 0.00379251 0.00363475 0.00351623 0.00332765 0.00315913 0.00294825
 0.00291903 0.00288763 0.00276924 0.00267501 0.00255198 0.0025103
 0.00246725 0.00234047 0.00225692 0.00219463 0.00214573 0.00211553
 0.00204214 0.00201227 0.00195361 0.00190866 0.00187676 0.00185268
 0.001753   0.0017259  0.00167419 0.00163022 0.0015972  0.0015514
 0.00154411 0.00151031 0.00146084 0.00144474 0.00143006 0.00140796
 0.00136761 0.00134241 0.00131593 0.00130933 0.00128266 0.00125402
 0.00120712 0.00119282 0.00117467 0.00116365 0.00113001 0.00111782
 0.00110435 0.00107277 0.00106339 0.00105238 0.00102334 0.0010085
 0.00099886 0.00099302 0.00097389 0.00095186 0.00094861 0.0009368

In [29]:
cumvar=0.0
for x in pca.explained_variance_ratio_:
    cumvar+=x
print(cumvar)

0.8751295162480821


In [30]:
print(pca.components_.shape)

(144, 307200)


In [31]:
import pickle

In [32]:
# save the model to disk
filename = 'sfdd_pca_fv1.pkl'
pickle.dump(pca, open(filename, 'wb'))

In [33]:
for itm in train1:
    Ytrain1.append(int(itm[1][1]))
    SubjectTrain1.append(itm[0])

In [34]:
Ytrain1NP=np.array(Ytrain1)

In [35]:
SubjectTrain1NP=np.array(SubjectTrain1)

In [36]:
# Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(principalComponents,Ytrain1NP)
print(model)
# make predictions
expected = Ytrain1NP
predicted = model.predict(principalComponents)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
cm = metrics.confusion_matrix(expected, predicted)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       142
          1       1.00      1.00      1.00       122
          2       1.00      1.00      1.00       110
          3       1.00      1.00      1.00       110
          4       1.00      1.00      1.00       129
          5       1.00      1.00      1.00       104
          6       1.00      1.00      1.00       134
          7       1.00      1.00      1.00        93
          8       1.00      1.00      1.00        73
          9       1.00      1.00      1.00       104

avg / total       1.00      1.00      1.00      1121

[[142   0  

In [37]:
def getTransformedData(filename):
    if hasThisFile(filename):
        img=Image.open(filename)
        img=img.convert('L')
        tph=pca.transform([list(img.getdata())])
        tph2=list(tph[0]) 
        return tph2

In [38]:
#del [[SubjectTrain1NP,SubjectTrain1,principalComponents,Ytrain1,Ytrain1NP,Xtrain1NP]]
#gc.collect()

0

In [39]:
# there are 21303 traing data left , trained by 1121 imges to get pca
clsrp=[]
cfsrp=[]
basetd=train2
for i in range(2):
    currtd,latertd=train_test_split(basetd, test_size=0.95)
    pchld=[]
    yhld=[]
    for itm in currtd:  
        pchld.append(getTransformedData(trainImgFileDir+itm[1]+'/'+itm[2]))
        yhld.append(int(itm[1][1]))
    pchldNP=np.array(pchld)
    yhldNP=np.array(yhld)
    # Decision Tree Classifier make predictions
    expected = yhldNP
    predicted = model.predict(pchldNP)
    rp1=metrics.confusion_matrix(expected, predicted)
    rp2=metrics.classification_report(expected, predicted)
    cfsrp.append(rp1)
    clsrp.append(rp2)
    del [[currtd,basetd,pchldNP,yhldNP,pchld,yhld,expected,predicted]]
    gc.collect()
    basetd=latertd

In [40]:
print(len(basetd))

19227


In [41]:
fif=open('sfdd_train_data_rest_fv1.csv','w')
for x in basetd:
    fif.write(','.join(x)+'\n')
fif.close() 

In [42]:
print(cfsrp[0])
print('\n')
print(cfsrp[1])

[[59 12  4  8  7  2 11  1  5 11]
 [ 2 58  5  7 10  8  8  2  7  3]
 [ 7  2 64  8  4  3 10  8  1  5]
 [ 3  3  2 63  8  1  5  1  3  6]
 [ 4  7 11 11 58 11  6  7  2  4]
 [ 7  1  1  1  6 65  9  2  4  1]
 [ 7 11 12  9 10  6 64  3  6  3]
 [ 6  3  7  7  5  7  6 39  5  5]
 [ 3  5  9  8 13  6  8  5 18 12]
 [ 6  7  4  6  5  6  6  1  5 56]]


[[54 10  4  8  5  7  7  3  7 11]
 [ 6 61  7  3 14  5  6  3  5  4]
 [ 3  3 67  4  5  2 10  7  7  6]
 [ 5  7  5 47  5  7  9  1  4  7]
 [ 5  6  4 12 51  6  4  4  2  3]
 [ 5  5  4  3  4 53  9  3  0  4]
 [ 4  9  7  4  7  2 48  5  5  3]
 [ 4  4  5  6  3  4 10 52  8  7]
 [ 7  5  4  5 12  9 12  8 21  8]
 [ 5  7  2 11  4  3  7  3  4 49]]


In [43]:
print(clsrp[0])
print(clsrp[1])

             precision    recall  f1-score   support

          0       0.57      0.49      0.53       120
          1       0.53      0.53      0.53       110
          2       0.54      0.57      0.55       112
          3       0.49      0.66      0.57        95
          4       0.46      0.48      0.47       121
          5       0.57      0.67      0.61        97
          6       0.48      0.49      0.48       131
          7       0.57      0.43      0.49        90
          8       0.32      0.21      0.25        87
          9       0.53      0.55      0.54       102

avg / total       0.51      0.51      0.51      1065

             precision    recall  f1-score   support

          0       0.55      0.47      0.50       116
          1       0.52      0.54      0.53       114
          2       0.61      0.59      0.60       114
          3       0.46      0.48      0.47        97
          4       0.46      0.53      0.49        97
          5       0.54      0.59      0.56